In [1]:
# Calculation for PTRACER
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

In [43]:
hs = 1
# /Volumes/Extreme SSD/MITgcm outputs/ & ../Model_outputs
tracer = xr.open_dataset('../Model_outputs/TracerAge/ptracers_' + str(format(hs,'03d')) + '.nc')
#tracer2 = xr.open_dataset('../Model_outputs/Tracer01/ptracers_' + str(format(hs,'03d')) + '.nc')
state = xr.open_dataset('../Model_outputs/TracerAge/state_' + str(format(hs,'03d')) + '.nc')
Grid = xr.open_dataset('../Model_outputs/TracerAge/grid_' + str(format(hs,'03d')) + '.nc')

HFacC = Grid.HFacC.data
drF = Grid.drF.data
rA = Grid.rA.data
cv = np.broadcast_to(rA[np.newaxis, :, :], HFacC.shape) * np.broadcast_to(drF[:, np.newaxis, np.newaxis], HFacC.shape) * HFacC

time = state.T.data
x = state.X.data
z = state.Z.data

tage1 = tracer.tr1_age.data
#tr1 = tracer.tr1.data
#tr1_rlx1 = tracer.tr1_rlx.data
#tr1_rlx2 = tracer2.tr1_rlx.data
rA.shape

(69, 175)

In [ ]:
# Volume weighted concentration

Tr1 = np.empty(len(time))
Tr1_rlx1 = np.empty(len(time))
Tr1_rlx2 = np.empty(len(time))

for i in range(len(time)):
    Tr1[i] = np.sum(tr1[i,40:,:,120]*cv[40:,:,120]) / cv[40:,:,120].sum()
    Tr1_rlx1[i] = np.sum(tr1_rlx1[i,40:,:,120]*cv[40:,:,120]) / cv[40:,:,120].sum()
    Tr1_rlx2[i] = np.sum(tr1_rlx2[i,40:,:,120]*cv[40:,:,120]) / cv[40:,:,120].sum()

In [ ]:
fig1,ax1 = plt.subplots(figsize=(10,7))

ax1.plot(time,Tr1,'-',label='Initial(no relax)')
ax1.plot(time,Tr1_rlx1,'-',label='Relax-100s')
ax1.plot(time,Tr1_rlx2,'-',label='Relax-1800s')

ax1.legend(loc='best', fontsize=15)
ax1.set_ylabel(r'Concentration',size = 20)
ax1.set_xlabel(r'Time (s)', size = 20)
ax1.grid(linestyle='--', linewidth='0.5')
ax1.set(xlim=(0, time[-1]), ylim=(0.1, 1))
#plt.title(r'Relaxing Timescale = 1800s',size=20)

In [ ]:
# Estimate the age of water parcel (Rayson et al., 2016)

# Average in y direction
tr1_zx = tr1.mean(2)
tr1rlx_zx = tr1_rlx1.mean(2)
dt = time[1] - time[0]

NT, NZ, NX = tr1_zx.shape
alpha = np.zeros((NT, NZ, NX)) # age concentration
C = tr1_zx # total concentration

for j in range(1,len(time)-1):
    alpha[1,:,:] = tr1rlx_zx[1,:,:] * dt
    alpha[j+1,:,:] = alpha[j] + tr1rlx_zx[j,:,:] * dt

#for j in range(len(time)):
#    alpha[j,:,:] = tr1rlx_zx[j,:,:] * time[j]    
    
    
# Mean age of the water parcel
a = alpha / C
#a[np.isnan(a)] == 0
A = np.nan_to_num(a)
#np.nanmax(a), np.nanmin(a), np.nanmean(a)

In [ ]:
A[50,:,118]

In [ ]:
# Contour plot of the age
cm = np.linspace(0,1e4,20) # Contour range

cmap = 'RdBu_r'


t1 = -1
xl1, xr1 = 100, 123

fig2 = plt.figure(figsize=(20,5))
fig2.tight_layout()  
       
ax1 = fig2.add_subplot(111)
im1 = ax1.contourf(x[xl1:xr1], z, A[t1,:,xl1:xr1], cm, cmap=cmap,extend='max')
ax1.axvline(x=x[120], color='g',lw=2)


cbar1 = fig2.colorbar(im1)
cbar1.set_ticks([0, .5e4, 1e4])
cbar1.ax.set_title(r'Age (s)', size=12)

# Set Colorbar
#cbaxes = fig2.add_axes([0.75, .9, 0.15, 0.02])
#cbar = fig2.colorbar(im1,cax=cbaxes, orientation='horizontal')
#cbar.set_ticks([-5, 0, 5])
#cbar.ax.set_title(r'Diffusive Heat $(\times{10^{-5}}^\circ$C/s)', size=12)

ax1.set_ylabel('Depth (m)', size=15)
#ax1.set_title(r'(b) Initial', loc='left', size=15, weight='bold')
ax1.set_xlabel('Distance from glacier (m)', size=15)


In [ ]:
# Contour plot of the tracer concentration
cm1 = np.linspace(0,1,20) # Contour range
cm2 = np.linspace(0,.2,20) # Contour range
cmap = 'RdBu_r'

t2 = -1
xl2, xr2 = 100, 123

fig3 = plt.figure(figsize=(18,9))
fig3.tight_layout()  
       
ax1 = fig3.add_subplot(211)
im1 = ax1.contourf(x[xl2:xr2], z, tr1rlx_zx[t2,:,xl2:xr2], cm1, cmap=cmap)
ax1.axvline(x=x[120], color='g',lw=2)

ax1.set_ylabel('Depth (m)', size=15)
ax1.xaxis.set_visible(False)
#ax1.set_xlim([x[Xl], x[Xr-1]])
ax1.set_facecolor('k')
ax1.set_title(r'(a) Source', loc='left', size=15, weight='bold')

# Set Colorbar

#plt.clabel(im1, fmt='%2.1f', fontsize=14)
#plt.colorbar(im1)

#cbaxes1 = fig3.add_axes([0.75, .91, 0.15, 0.02])
cbar1 = fig3.colorbar(im1)
cbar1.set_ticks([0, .5, 1])
cbar1.ax.set_title(r'Concentration', size=12)

       
ax2 = fig3.add_subplot(212)
im2 = ax2.contourf(x[xl2:xr2], z, tr1_zx[t2,:,xl2:xr2], cm2, cmap=cmap)
ax2.axvline(x=x[120], color='g',lw=2)

cbar2 = fig3.colorbar(im2)
cbar2.set_ticks([0, .1, .2])
cbar2.ax.set_title(r'Concentration', size=12)
#cbaxes2 = fig3.add_axes([0.75, .49, 0.15, 0.02])
#cbar2 = fig3.colorbar(im2,cax=cbaxes2, orientation='horizontal')
#cbar2.set_ticks([0,.04,.08])
#cbar2.ax.set_title(r'Concentration', size=12)

ax2.set_ylabel('Depth (m)', size=15)
ax2.set_title(r'(b) Initial', loc='left', size=15, weight='bold')
ax2.set_xlabel('Distance from glacier (m)', size=15)


In [41]:
cm4 = np.linspace(0,2,20) # Contour range
cmap4 = 'RdBu_r'

tage = tage1.mean(2)
xl2, xr2 = 105, 125

fig_path = "/Users/weiyangbao/Documents/Results_Figs/PTRACERS/Age/"
if not os.path.exists(fig_path):
    os.makedirs(fig_path)

for t in range(len(time)):
    fig4 = plt.figure(figsize=(24,4))
    fig4.tight_layout()  
       
    ax4 = fig4.add_subplot(111) 
    im4 = ax4.contourf(x[xl2:xr2], z, np.log10(tage[t,:,xl2:xr2]), cm4, cmap=cmap4)
    ax4.axvline(x=x[120], color='g',lw=2)

    ax4.set_ylabel('Depth (m)', size=15)
    ax4.set_xlabel('Distance from glacier (m)', size=15)

    #ax4.xaxis.set_visible(False)
    #ax1.set_xlim([x[Xl], x[Xr-1]])
    ax4.set_facecolor('k')
    ax4.set_title(r'Time: '+str(t*100)+' s', loc='center', size=15, weight='bold')
    
    # Set Colorbar

    #plt.clabel(im1, fmt='%2.1f', fontsize=14)
    #plt.colorbar(im1)

    #cbaxes1 = fig3.add_axes([0.75, .91, 0.15, 0.02])
    cbar4 = fig4.colorbar(im4)
    cbar4.set_ticks([0, .5, 1, 1.5, 2])
    cbar4.ax.set_title(r'Concentration ($\log_{10}$)', size=12)
 
    
    plt.close()
    fig4.savefig(fig_path + '/time_' + format(t, '03d') + '.png', dpi=150, bbox_inches='tight')

/Users/weiyangbao/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log10
  app.launch_new_instance()


In [42]:
# Generate GIF from saved output figures
import imageio
import glob

images = []
imagenames = sorted(glob.glob(fig_path + '/time_*.png'))

for filename in imagenames:
    images.append(imageio.imread(filename))
imageio.mimsave(fig_path + 'TracerAge.gif', images, duration=.2)

In [ ]:
fig_path = "/Users/weiyangbao/Documents/Results_Figs/PTRACERS/"
if not os.path.exists(fig_path):
    os.makedirs(fig_path)

#fig1.savefig(fig_path + '/Rlx_scale.png', dpi=200, bbox_inches='tight')
fig3.savefig(fig_path + '/Concentration.png', dpi=200, bbox_inches='tight')

In [36]:
np.arange(0,101,2)

array([  0,   2,   4,   6,   8,  10,  12,  14,  16,  18,  20,  22,  24,
        26,  28,  30,  32,  34,  36,  38,  40,  42,  44,  46,  48,  50,
        52,  54,  56,  58,  60,  62,  64,  66,  68,  70,  72,  74,  76,
        78,  80,  82,  84,  86,  88,  90,  92,  94,  96,  98, 100])